# Instructions
The following code was designed in order to load in and score motion/freezing data from video files. This code should be run with carefully selected parameters using the more extensive visualization options provided in FreezeAnalyis.ipynb.  In addition to saving frame by frame motion/freezing information for each video in separate csv files, bins can also be defined for summarizing videos (e.g. minute by minute).  All summary information will be saved in a single file.  If no bins are defined, the code will automatically create a summary file that lists average session motion/freezing per video.

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

---
# 1. Load Necessary Packages

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BehaviorTracking_Functions as fz
plt.gray()

---
# 2. User Sets Directory and Parameters for Motion/Freeze Analysis
Note that all videos of particular type will be analyzed.  Videos should be of the same length if user is defining bins.  Videos should be the same fps and the same width/height.

In [ ]:
# directory containing all video files to be processed
dpath = "/Users/ZP/Desktop/Videos"

#video parameters
ftype = "mpg" #file type.  so far only mpg1 ("mpg") and wmv ("wmb") have been tested
fps = 30 #fps of video files
ycrop = 150 #cropping of video frame
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 

#Motion/Freezing parameters
mt_cutoff = 10.3 #Motion cutoff
FreezeThresh = 188 #Upper bound for freezing, in frame-by-frame pixels changed
MinDuration = .5 #Number of seconds motion must be below threshold to beging accruing freezing


---
# 3. User Defines Bins for Summary data

In [ ]:
Use_Bins = True #Define whether user specified summary bins are to bew used.  True/False.

#USER SETS BIN INFORMATION
Bin_Names = ['30','60','90','120','150','180','210','240','270','300'] #Create list of bin names. Must be in single/double quotes (eg: ['avg','1','2'])
Bin_Start = np.arange(0,300,30) #provide list of bin start times, in seconds (eg: [0,0,60])
Bin_Stop = np.arange(20,301,30) #provide list of bin end times, in seconds (eg: [120,60,120])

#Check requirement that lists be equal length
fz.Check(Bin_Names,Bin_Start,Bin_Stop)  

---
# 4. Analyze Videos

In [ ]:
fz.Batch(dpath,ftype,fps,ycrop,SIGMA,mt_cutoff,FreezeThresh,MinDuration,Use_Bins,Bin_Names,Bin_Start,Bin_Stop)